### CONSIGNA: Clasificadores de Texto - Attention
Generar un modelo que como mínimo consista en un mecanismo de attention con un contextualizador convolucional. Ajustar los hiperparámentros para mejorar el accuracy. Se pueden extraer ideas de otros modelos como el TextCNN. Cambiar el contextualizador a una Bidir LSTM y comparar tiempos de entrenamiento y performance.

* No se encuentra una notebook subida con el codigo de la cátedra ya que lo que es interesante para este TP se encuentra en esta notebook, mientras que las complicaciones que traia aparejada la tarea de correr el repo se resolvio durante la clase.
Por otro lado, cabe destacar la diferencia entre correrlo en collab con el GPU a correrlo en vscode en mi compu. El tiempo fue abismal.
* No tenia gensim y el enlace a drive no andaba, se empleo entonces los embeddings preentrenados de glove. En clase se comento la posibilidad de emplear fasttext tambien pero se prefiere mantener glove para mantener una comparacion frente a otros compañeros.
* En cuanto al tp: la idea principal de usar attention es que si bien uno tiene acceso a todo, en vez de ponderarlos de la misma forma, aplicas un criterio. Este criterio es aplicado por una sub-red dentro de nuestro modelo (en el caso de la bidir empleamos LSTM). Esta sub-red se encarga de estimar parámetros en funcion de t-1 y los estados actuales.
* La performance de accuracy aumenta más rápido al emplear CNN con attention que en BIDIR, en BIDIR tiene un aumento mas proporcional al pasar los epochs. Sin embargo, pese a que puede ser una complicación de la resolución llevada a cabo por mi, CNN parece overfitear mucho mas rápido que BIDIR.
* A destacar, el BIDIR se estanca en 0.5 mientras que la misma red llega a 0.8 cambiando glove por fasttext. Supongo esto tiene que ver con la naturalidad del embedding descargado, el de glove proviene de wikipedia mientras que el de fasttext de Facebook.
* En cuanto a tiempos de entrenamiento, las capas convolucionales de CNN en la notebook hacian parecer a la compu como un auto de f1; mientras que en google collab no se aprecia una diferencia clara respecto al costo de tiempo de entrenamiento.
* Si bien se recomendaba usar TEXTCNN se opto por la red 5 del repositorio ya que englobaba muchas cosas.
* El tiempo de entrenamiento de la red es similar, pero la bidir necesita muchas más epochs para converger (pero sus epochs son mucho mas cortas que la red convolucional)
################# TO-DO Escribir bien las conclusiones aunque ya se hablo todo en clase




In [1]:
from sklearn.datasets import fetch_20newsgroups
from matplotlib import pyplot as plt
from collections import Counter
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Attention, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Lambda
from keras.models import Sequential,Model
from keras import optimizers
import keras.backend as K


In [2]:
token=Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token="UNK", document_count=0)

In [3]:
token.fit_on_texts(newsgroups_train.data)

In [4]:
train_sequences=token.texts_to_sequences(newsgroups_train.data)
test_sequences=token.texts_to_sequences(newsgroups_test.data)


In [5]:
max_len=500
train_sequences=pad_sequences(train_sequences,maxlen=max_len)
test_sequences=pad_sequences(test_sequences,maxlen=max_len)

In [6]:
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])

In [8]:
glove_path = 'glove.6B.300d.txt'


embeddings_index = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs






In [9]:
# Define the dimensions
num_words = len(dictionary) + 1  # Add 1 for the padding token (if used)
embed_dim = 300  # Change this to match the dimension of your GloVe embeddings

# Create an embedding matrix for your vocabulary
embedding_matrix = np.zeros((num_words, embed_dim))

# Fill the embedding matrix with GloVe embeddings
for word, idx in dictionary.items():
    if idx < num_words and word in embeddings_index:
        embedding_vector = embeddings_index[word]
        embedding_matrix[idx] = embedding_vector
    else:
        # Handle out-of-vocabulary words here (e.g., use random initialization or zeros)
        pass

In [10]:
import tensorflow as tf
K.clear_session()

In [ ]:
from keras.layers import Embedding, Conv1D, Dense, Input, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, MaxPooling1D, GlobalMaxPooling1D, Dropout, Reshape
import keras.backend as K
from keras.activations import softmax
from keras.models import Model
from keras import optimizers
value_dim=100
def softMaxOverTime(x):
    return softmax(x,axis=1)

nb_words=num_words
num_filters=64
input_layer = Input(shape=(max_len,))
embedding_layer=Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)(input_layer)
conv_out=Conv1D(value_dim,8,padding="same")(embedding_layer)
conv_out=Activation("relu")(conv_out)
conv_out=Conv1D(value_dim,8,activation="elu",padding="same")(conv_out)
ulog_attention=Dense(1,activation="linear")(conv_out)
attention=Activation(softMaxOverTime)(ulog_attention)
repeated_attention=TimeDistributed(RepeatVector(value_dim))(attention)
repeated_attention=Reshape([max_len,value_dim])(repeated_attention)
weighted_embeddings=Multiply()([repeated_attention,conv_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)
dense1=Dense(1000, activation='relu')(embedding_sum)
dense1_2=Dense(100, activation='relu')(dense1)
dense2=Dense(20, activation='softmax')(dense1_2)
model=Model(input_layer , dense2)
adam = optimizers.Adam(lr=0.001, beta_1=0.999, beta_2=0.999, epsilon=1e-09, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

c:\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 300)     40243200    ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 100)     240100      ['embedding[0][0]']              
                                                                                                  
 activation (Activation)        (None, 500, 100)     0           ['conv1d[0][0]']                 
                                                                                              

In [ ]:
model.fit(train_sequences, newsgroups_train.target,batch_size=250,epochs=20,validation_split=0.2)

Epoch 1/20
37/37 [==============================] - 120s 3s/step - loss: 2.7110 - accuracy: 0.1429 - val_loss: 2.1725 - val_accuracy: 0.2545
Epoch 2/20
37/37 [==============================] - 108s 3s/step - loss: 1.9132 - accuracy: 0.3518 - val_loss: 1.6932 - val_accuracy: 0.4238
Epoch 3/20
37/37 [==============================] - 116s 3s/step - loss: 1.5229 - accuracy: 0.4671 - val_loss: 1.3856 - val_accuracy: 0.5214
Epoch 4/20
37/37 [==============================] - 105s 3s/step - loss: 1.2392 - accuracy: 0.5607 - val_loss: 1.1926 - val_accuracy: 0.5802
Epoch 5/20
37/37 [==============================] - 98s 3s/step - loss: 1.0701 - accuracy: 0.6197 - val_loss: 1.1095 - val_accuracy: 0.6151
Epoch 6/20
37/37 [==============================] - 100s 3s/step - loss: 0.9475 - accuracy: 0.6593 - val_loss: 1.0564 - val_accuracy: 0.6403
Epoch 7/20
37/37 [==============================] - 98s 3s/step - loss: 0.8642 - accuracy: 0.6898 - val_loss: 0.9577 - val_accuracy: 0.6792
Epoch 8/20
37/3

In [ ]:
model.save("CNN+ATTENTION.h5")


In [106]:
K.clear_session()

In [107]:
from keras.layers import Embedding, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Bidirectional, LSTM
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input
from keras.layers import Embedding, Conv1D, Dense, Input, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, MaxPooling1D, GlobalMaxPooling1D, Dropout, Reshape
import keras.backend as K
from keras.activations import softmax
from keras.models import Model
from keras import optimizers
value_dim=100

def softMaxOverTime(x):
    return softmax(x,axis=1)

nb_words=num_words
num_filters=64
input_layer = Input(shape=(max_len,))
embedding_layer=Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)(input_layer)
lstm_out=Bidirectional(LSTM(value_dim, return_sequences=True,activation="tanh"),merge_mode="sum")(embedding_layer)
conv_out=Conv1D(value_dim,64,padding="same")(lstm_out)
conv_out=Activation("linear")(conv_out)
conv_out=Conv1D(value_dim,64,activation="linear",padding="same")(conv_out)
ulog_attention=Dense(1,activation="linear")(conv_out)
attention=Activation(softMaxOverTime)(ulog_attention)
repeated_attention=TimeDistributed(RepeatVector(value_dim))(attention)
repeated_attention=Reshape([max_len,value_dim])(repeated_attention)
weighted_embeddings=Multiply()([repeated_attention,lstm_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)
dense1=Dense(2000, activation='relu')(embedding_sum)
dense2=Dense(200, activation='softmax')(dense1)
model=Model(input_layer , dense2)
model.compile(loss='sparse_categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

In [108]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 500)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 500, 300)             4024320   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 bidirectional (Bidirection  (None, 500, 100)             320800    ['embedding[0][0]']           
 al)                                                                                              
                                                                                              

In [109]:
model.fit(train_sequences, newsgroups_train.target,batch_size=32,epochs=50,validation_split=0.2)

Epoch 1/50


Exception ignored in: <function _xla_gc_callback at 0x7877342cb490>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


283/283 [==============================] - 161s 69ms/step - loss: 2.9856 - accuracy: 0.0872 - val_loss: 2.8068 - val_accuracy: 0.0897
Epoch 2/50
283/283 [==============================] - 20s 70ms/step - loss: 2.7039 - accuracy: 0.1435 - val_loss: 2.5413 - val_accuracy: 0.1617
Epoch 3/50
283/283 [==============================] - 19s 66ms/step - loss: 2.5297 - accuracy: 0.1775 - val_loss: 2.4502 - val_accuracy: 0.2028
Epoch 4/50
283/283 [==============================] - 19s 66ms/step - loss: 2.4699 - accuracy: 0.2052 - val_loss: 2.3710 - val_accuracy: 0.2360
Epoch 5/50
283/283 [==============================] - 19s 66ms/step - loss: 2.3680 - accuracy: 0.2318 - val_loss: 2.2464 - val_accuracy: 0.2391
Epoch 6/50
283/283 [==============================] - 19s 66ms/step - loss: 2.2097 - accuracy: 0.2685 - val_loss: 2.1273 - val_accuracy: 0.2863
Epoch 7/50
283/283 [==============================] - 19s 68ms/step - loss: 2.1145 - accuracy: 0.2981 - val_loss: 2.1473 - val_accuracy: 0.3014
Ep

KeyboardInterrupt: ignored

In [ ]:
model.save("BIDIR+ATTENTION.h5")
K.clear_session()

Helpers producto de redes1 por si necesito usar en algun momento

In [ ]:
K.clear_session()

In [ ]:
model.save("modelo.h5")

In [ ]:
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
model.fit(train_x, train_y,batch_size=28,epochs=30,validation_split=0.2,callbacks=[early_stopping])

In [ ]:
txt_path = 'path/to/x.txt'
word_to_vec = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        word_to_vec[word] = vector